In [1]:
import numpy as np
import pandas as pd
from pyecharts import Kline, Overlap, Line, Grid, EffectScatter
%matplotlib inline

In [2]:
from LayerModel5 import *

In [3]:
from frame import *

In [4]:
class Pattern01(PatternPair):
    
    def calChart(self):
        fd0 = self.obj_list[0].calFeatures()
        fd1 = self.obj_list[1].calFeatures()
        
        is_flag0 = self.is_flag(fd0)
        
        is_flag1 = self.is_flag(fd1)
        is_synthetic = eq(choose(fd0, 'drt'), choose(fd1, 'drt'))
        is_move = self.real_move(fd0, fd1)
        little_flag = le(choose(fd1, 'a'), 0.5)
        
        r = All([is_flag0, is_flag1, is_synthetic, is_move, little_flag])
        v = NodeVisitor()
        return v.evaluate(r)      


         

### import from mongoDB

In [5]:


from pymongo import MongoClient, ASCENDING
mc = MongoClient()                  # Mongo连接
dbMinute30 = mc['Trade2019_Minute30']       # 数据库

symbol = '600030'
TmS = '2018-10-01 10:30'
#TmE = '2019-04-04 10:30'
monthS = 3
cl = dbMinute30[symbol]
#flt = {'month':{'$gte':monthS}}  
flt = {}
dataCursor = cl.find(flt, {'_id':0})
# sort by datetime
# to learn method of MongoDB

dt = [[d['open'], d['high'], d['low'], d['close'], d['datetime']] for d in dataCursor]
dt_df = pd.DataFrame(dt, columns= ['open','high','low','close','datetime'])


In [6]:
dt_df.tail()

,open,high,low,close,datetime
1597,17.50,17.61,17.50,17.60,2019-01-16 11:30
1598,17.59,17.64,17.56,17.62,2019-01-16 13:30
1599,17.62,17.71,17.62,17.70,2019-01-16 14:00
1600,17.70,17.74,17.64,17.69,2019-01-16 14:30
1601,17.69,17.70,17.63,17.64,2019-01-16 15:00


In [7]:
dt_df.sort_values(by=['datetime'], inplace=True)
dt_df.reset_index(drop=True,inplace=True)
dt_df['TmIdx'] = dt_df.index

In [8]:
dt = dt_df[['open','high','low','close','TmIdx']].values.tolist()
x = dt_df[['datetime','TmIdx']]

In [9]:
#x.head()

In [10]:
def main():
    global dt,m,ef
    layer = m.layer
    
    Event.L = [[] for i in list(range(layer+1))]
    #filepath = 'event_config.json'
    #ef = EventFactory(filepath)
    
    for k in dt[1:]:
        m.update(k)
        Stick.lv_L[-1].update(k)  
        
        # update crt Lv1 and add new Lv1
        TrendLv1.lv_L[-1].updateEndP(k)
        for i in list(range(layer)):
            if len(Event.L[i]) > 0:
                print('TmIdx:{},[{}]'.format(k[4],[x for x in Event.L[i]]))
                for event in Event.L[i][:]:
                    actions = ef.play(event)
                    for a in actions:
                        print(a)
                        if a != '':
                            eval(a)
             
    return None

In [11]:
m = Market(4)
Stick.mm = m
Trend.m = m
Pair.m = m
PairChain.m = m

In [12]:
filepath = 'event_config.json'
ef = EventFactory(filepath)

In [13]:
Stick.lv_L = m.Lv0_L
Stick('init', k_bar=dt[0])

Stick(drt:0, lv:0, Point(0.0, 16.31, 0))

In [14]:
TrendLv1 = type('TrendLv1', (Trend,), {'lv_L':m.Lv1_L, 'ML':Stick.lv_L, 'level':1}) 
TrendLv1('init', k_bar=dt[0], mp=[0])
TrendLv2 = type('TrendLv2', (Trend,), {'lv_L':m.Lv2_L, 'ML':TrendLv1.lv_L, 'level':2}) 
TrendLv2('init', k_bar=dt[0], mp=[0])
TrendLv3 = type('TrendLv3', (Trend,), {'lv_L':m.Lv3_L, 'ML':TrendLv2.lv_L, 'level':3}) 
TrendLv3('init', k_bar=dt[0], mp=[0])

Trend3(0, 0, Point(0.0, 16.31, 0))

In [15]:
PairLv0 = type('PairLv0', (Pair,), {'ML': m.findList('st', 0), 'L': m.findList('pair', 0), 'level': 0})
PairLv0(0)
PairLv1 = type('PairLv1', (Pair,), {'ML': m.findList('st', 1), 'L': m.findList('pair', 1), 'level': 1})
PairLv1(0)
PairLv2 = type('PairLv2', (Pair,), {'ML': m.findList('st', 2), 'L': m.findList('pair', 2), 'level': 2})
PairLv2(0)
PairLv3 = type('PairLv3', (Pair,), {'ML': m.findList('st', 3), 'L': m.findList('pair', 3), 'level': 3})
PairLv3(0)
PairChainLv0 = PairChain(0, 'PairChainLv0')
PairChainLv0.regSignal()
PairChainLv1 = PairChain(1,'PairChainLv1')
PairChainLv1.regSignal()
PairChainLv2 = PairChain(2, 'PairChainLv2')
PairChainLv2.regSignal()
PairChainLv3 = PairChain(3, 'PairChainLv3')


In [16]:
class Pattern01Lv0(Pattern01):
    def __init__(self,cursor):   
        super().__init__()
        self.changeL(m.findList('pattern01', 0))
        self.L.append(self)
        self.level = 0
        self.chain_layer = cursor[0]
        self.changeML(m.findList('pairchain', 0))
        self.cursor = cursor
        self.changeW(2)      


In [17]:
class Pattern01Lv1(Pattern01):
    def __init__(self,cursor):   
        super().__init__()
        self.changeL(m.findList('pattern01', 1))
        self.L.append(self)
        self.level = 1
        self.chain_layer = cursor[0]
        self.changeML(m.findList('pairchain', 1))
        self.cursor = cursor
        self.changeW(2)      

In [18]:
class Pattern01Lv2(Pattern01):
    def __init__(self,cursor):   
        super().__init__()
        self.changeL(m.findList('pattern01', 2))
        self.L.append(self)
        self.level = 2
        self.chain_layer = cursor[0]
        self.changeML(m.findList('pairchain', 2))
        self.cursor = cursor
        self.changeW(2)      

In [19]:
PatternPair.m = m
Pattern01Lv0([0,0])
Pattern01Lv0([1,0])
Pattern01Lv0([2,0])
m.PT01Lv0_L[0].regAction()
Pattern01Lv1([0,0])
Pattern01Lv1([1,0])
Pattern01Lv1([2,0])
m.PT01Lv1_L[0].regAction()
Pattern01Lv2([0,0])
Pattern01Lv2([1,0])
Pattern01Lv2([2,0])
m.PT01Lv2_L[0].regAction()

In [20]:
m.__dict__

{'CenterStrict_LD': {},
 'Lv0_L': [Stick(drt:0, lv:0, Point(0.0, 16.31, 0))],
 'Lv1_L': [Trend1(0, 0, Point(0.0, 16.31, 0))],
 'Lv2_L': [Trend2(0, 0, Point(0.0, 16.31, 0))],
 'Lv3_L': [Trend3(0, 0, Point(0.0, 16.31, 0))],
 'MPLv0_L': [],
 'MPLv1_L': [],
 'MPLv2_L': [],
 'MPLv3_L': [],
 'PCLv0_L': [<LayerModel5.PairChain at 0x23722d54668>],
 'PCLv1_L': [<LayerModel5.PairChain at 0x237230322b0>],
 'PCLv2_L': [<LayerModel5.PairChain at 0x23723005128>],
 'PCLv3_L': [<LayerModel5.PairChain at 0x23723005160>],
 'PLv0_L': [Pair-1(TmS:0.0, 0, [0])],
 'PLv1_L': [Pair-1(TmS:0.0, 0, [0])],
 'PLv2_L': [Pair-1(TmS:0.0, 0, [0])],
 'PLv3_L': [Pair-1(TmS:0.0, 0, [0])],
 'PT01Lv0_L': [<__main__.Pattern01Lv0 at 0x23723032e48>,
 'PT01Lv1_L': [<__main__.Pattern01Lv1 at 0x23723032400>,
 'PT01Lv2_L': [<__main__.Pattern01Lv2 at 0x237230322e8>,
 'PT01Lv3_L': [],
 'TmIdx': 0,
 'bin_cnt': 20,
 'layer': 4,
 'obj_list': {'mergedpair': [[], [], [], [], []],
  'pair': [[Pair-1(TmS:0.0, 0, [0])],
   [Pair-1(TmS:0.0,

In [21]:
ef.event_config

[{'Pair': {'NEW': {'method': ['updateAll'],
    'obj_p': ['m.PT01Lv0_L'],
    'param': ['m.PLv0_L[-1]']}},
  'PairChainLv0': {'NEW': {'method': ['updateAll'],
    'obj_p': ['Pattern01Lv0'],
    'param': ['']}},
  'Stick': {'LVUPD': {'method': ['updatePeakP'],
    'obj_p': ['m.Lv1_L[-1]'],
    'param': ['k_bar=m.dt[-1]']},
   'NEW': {'method': ['update2', 'update2'],
    'obj_p': ['m.Lv1_L[-1]', 'm.PCLv0_L[0]'],
    'param': ['', '']},
   'PEAK': {'method': ['updateLastSt'],
    'obj_p': ['m.PCLv0_L[0]'],
    'param': ['']}},
  'level': 0},
 {'Pair': {'NEW': {'method': ['updateAll'],
    'obj_p': ['m.PT01Lv1_L'],
    'param': ['m.PLv1_L[-1]']}},
  'PairChainLv1': {'NEW': {'method': ['updateAll'],
    'obj_p': ['Pattern01Lv1'],
    'param': ['']}},
  'TrendLv1': {'LVUPD': {'method': ['updatePeakP'],
    'obj_p': ['m.Lv2_L[-1]'],
    'param': ['k_bar=m.dt[-1]']},
   'NEW': {'method': ['update2', 'update2'],
    'obj_p': ['m.Lv2_L[-1]', 'm.PCLv1_L[0]'],
    'param': ['', '']},
   'PEAK': {

In [22]:
Position.m = m
Position.L = m.position

In [23]:
main()

TmIdx:15.0,[[<frame.Event object at 0x0000023722D54358>]]
(0, Stick, LVUPD)
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:17.0,[[<frame.Event object at 0x0000023722F5B6A0>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:0,treated.mp:[0]
Lv1.update2(): flag:0, treated.status:0,treaded.drt:1,treated.mp:[0, 1]
m.PCLv0_L[0].update2()
TmIdx:18.0,[[<frame.Event object at 0x0000023722F5BC18>, <frame.Event object at 0x0000023722F5B390>]]
(0, PairChainLv0, NEW)
Pattern01Lv0.updateAll()
test:PP.cursor()__start_cursor:[0, 0],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 0],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 0],layer_chain:2,w:2
(0, Stick, PEAK)
m.PCLv0_L[0].updateLastSt()
TmIdx:19.0,[[<frame.Event object at 0x0000023722F5BA20>]]
(0, Stick, PEAK)
m.PCLv0_L[0].updateLastSt()
TmIdx:20.0,[[<frame.Event object at 0x0000023722F5BDA0>]]
(0, Stick, PEAK)
m.PCLv0_L[0].updateLastSt()
TmIdx:21.0,[[<frame.Event object at 0x0000023722F5B898>]]
(0, S

m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[17, 18, 19]
TrendLv1.updatedPeakP(st_idx_in_ss):last_st_start:(drt:1, lv:0, (137.0, 15.9, 1)),peak(138.0, 15.92, 1),drt:-1
TrendLv1.updatedPeakP(st_idx_in_ss):FIND PEAK
Lv1.update2(): flag:0, treated.status:1,treaded.drt:-1,treated.mp:[17, 18, 19, 20]
m.PCLv0_L[0].update2()
m.position[1].open_position(m.dt[-1])
TmIdx:142.0,[[<frame.Event object at 0x00000237230D9710>]]
(0, PairChainLv0, NEW)
Pattern01Lv0.updateAll()
test:PP.cursor()__start_cursor:[0, 4],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0, 5],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 6],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 7],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 4],layer_chain:2,w:2
test:PP.cursor()__start_cursor:[2, 5],layer_chain:2,w:2
TmIdx:149.0,[[<frame.Event object at 0x00000237230C91D0>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[17, 18, 1

test:PP.cursor()__start_cursor:[2, 7],layer_chain:2,w:2
test:PP.cursor()__start_cursor:[2, 8],layer_chain:2,w:2
TmIdx:208.0,[[<frame.Event object at 0x00000237230D6588>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[25, 26, 27, 28, 29, 30]
Lv1.upd1st():case1, status:1, mp:[25, 26, 27, 28, 29, 30, 31],pp:3
Lv1.produceNewTrend:case1, mp=[28, 29, 30, 31]
TrendLv1.updatedPeakP(st_idx_in_ss):last_st_start:(drt:-1, lv:0, (195.0, 15.6, -1)),peak(196.0, 15.46, 1),drt:1
TrendLv1.updatedPeakP(st_idx_in_ss):FIND PEAK
TrendLv1.updatedPeakP(st_idx_in_ss):last_st_start:(drt:1, lv:0, (198.0, 15.42, 1)),peak(195.0, 15.6, -1),drt:1
TrendLv1.updatedPeakP(st_idx_in_ss):last_st_start:(drt:-1, lv:0, (205.0, 15.64, -1)),peak(195.0, 15.6, -1),drt:1
TrendLv1.updatedPeakP(st_idx_in_ss):FIND PEAK
Lv1.update2(): flag:2, treated.status:1,treaded.drt:1,treated.mp:[28, 29, 30, 31]
m.PCLv0_L[0].update2()
TmIdx:208.0,[[<frame.Event object at 0x00000237230D6DD8>]]
(1,

test:PP.cursor()__start_cursor:[1, 11],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 12],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 10],layer_chain:2,w:2
test:PP.cursor()__start_cursor:[2, 11],layer_chain:2,w:2
(0, Stick, PEAK)
m.PCLv0_L[0].updateLastSt()
TmIdx:249.0,[[<frame.Event object at 0x00000237230C4A20>]]
(0, Stick, LVUPD)
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:253.0,[[<frame.Event object at 0x00000237230EA550>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[33, 34, 35, 36, 37, 38]
Lv1.upd1st():case1, status:1, mp:[33, 34, 35, 36, 37, 38, 39],pp:3
Lv1.produceNewTrend:case1, mp=[36, 37, 38, 39]
TrendLv1.updatedPeakP(st_idx_in_ss):last_st_start:(drt:-1, lv:0, (230.0, 15.68, -1)),peak(231.0, 15.58, 1),drt:1
TrendLv1.updatedPeakP(st_idx_in_ss):FIND PEAK
TrendLv1.updatedPeakP(st_idx_in_ss):last_st_start:(drt:1, lv:0, (232.0, 15.52, 1)),peak(230.0, 15.68, -1),drt:1
TrendLv1.updatedPeakP(st_idx_in_ss):last_st_st

test:PP.cursor()__start_cursor:[1, 17],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 13],layer_chain:2,w:2
TmIdx:302.0,[[<frame.Event object at 0x00000237230EDA20>]]
(1, PairChainLv1, NEW)
Pattern01Lv1.updateAll()
test:PP.cursor()__start_cursor:[0, 2],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0, 3],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 0],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 1],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 0],layer_chain:2,w:2
TmIdx:302.0,[[<frame.Event object at 0x00000237230ED6A0>]]
(2, PairChainLv2, NEW)
Pattern01Lv2.updateAll()
test:PP.cursor()__start_cursor:[0, 1],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0, 2],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 0],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 1],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 0],layer_chain:2,w:2
TmIdx:304.0,[[<frame.Event object at 0x00000237230ED3C8>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_b

...

test:PP.cursor()__start_cursor:[1, 5],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 6],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 7],layer_chain:2,w:2
test:PP.cursor()__start_cursor:[2, 8],layer_chain:2,w:2
TmIdx:1416.0,[[<frame.Event object at 0x0000023723102D68>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[225, 226, 227, 228, 229, 230, 231, 232]
Lv1.update2(): flag:1, treated.status:1,treaded.drt:-1,treated.mp:[225, 226, 227, 228, 229, 230, 231, 232, 233]
m.PCLv0_L[0].update2()
TmIdx:1416.0,[[<frame.Event object at 0x00000237231180F0>]]
(1, TrendLv1, LVUPD)
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:1417.0,[[<frame.Event object at 0x0000023723118400>]]
(0, PairChainLv0, NEW)
Pattern01Lv0.updateAll()
test:PP.cursor()__start_cursor:[0, 72],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0, 73],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 80],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 81],layer_

test:PP.cursor()__start_cursor:[2, 9],layer_chain:2,w:2
TmIdx:1461.0,[[<frame.Event object at 0x00000237231142E8>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[235, 236, 237, 238, 239, 240]
Lv1.update2(): flag:2, treated.status:0,treaded.drt:1,treated.mp:[240, 241]
m.PCLv0_L[0].update2()
TmIdx:1461.0,[[<frame.Event object at 0x00000237231143C8>]]
(1, TrendLv1, NEW)
m.Lv2_L[-1].update2()
Lv2.update2()_begin: flag:0, treated.status:1,treated.mp:[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
TrendLv2.updatedPeakP(st_idx_in_ss):last_st_start:(1, (1450.0, 19.92, 1)),peak(1408.0, 19.46, 1),drt:-1
Lv2.update2(): flag:0, treated.status:1,treaded.drt:-1,treated.mp:[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
m.PCLv1_L[0].update2()
TmIdx:1462.0,[[<frame.Event object at 0x00000237231147F0>]]
(0, PairChainLv0, NEW)
Pattern01Lv0.updateAll()
test:PP.cursor()__start_cursor:[0, 74],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0

test:PP.cursor()__start_cursor:[0, 10],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 7],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 8],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 9],layer_chain:2,w:2
test:PP.cursor()__start_cursor:[2, 10],layer_chain:2,w:2
TmIdx:1491.0,[[<frame.Event object at 0x0000023723124550>]]
(2, PairChainLv2, NEW)
Pattern01Lv2.updateAll()
test:PP.cursor()__start_cursor:[0, 3],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0, 4],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 3],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 4],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 0],layer_chain:2,w:2
test:PP.cursor()__start_cursor:[2, 1],layer_chain:2,w:2
TmIdx:1493.0,[[<frame.Event object at 0x00000237231244E0>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[243, 244, 245, 246]
Lv1.upd1st():case1, status:1, mp:[243, 244, 245, 246, 247],pp:1
Lv1.produceNewTrend:case1, mp=[244

test:PP.cursor()__start_cursor:[0, 10],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 7],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 8],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 10],layer_chain:2,w:2
test:PP.cursor()__start_cursor:[2, 11],layer_chain:2,w:2
TmIdx:1543.0,[[<frame.Event object at 0x000002372312CE10>]]
(2, PairChainLv2, NEW)
Pattern01Lv2.updateAll()
test:PP.cursor()__start_cursor:[0, 4],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0, 5],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 4],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 5],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 1],layer_chain:2,w:2
TmIdx:1544.0,[[<frame.Event object at 0x000002372312C898>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[252, 253, 254, 255]
Lv1.update2(): flag:1, treated.status:1,treaded.drt:1,treated.mp:[252, 253, 254, 255, 256]
m.PCLv0_L[0].update2()
TmIdx:1544.0,[[<frame.Event object at 0

test:PP.cursor()__start_cursor:[0, 5],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 4],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 5],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[2, 1],layer_chain:2,w:2
TmIdx:1577.0,[[<frame.Event object at 0x0000023723136E10>]]
(0, Stick, NEW)
m.Lv1_L[-1].update2()
Lv1.update2()_begin: flag:0, treated.status:1,treated.mp:[259, 260, 261, 262]
Lv1.update2(): flag:1, treated.status:1,treaded.drt:-1,treated.mp:[259, 260, 261, 262, 263]
m.PCLv0_L[0].update2()
TmIdx:1577.0,[[<frame.Event object at 0x0000023723136908>]]
(1, TrendLv1, LVUPD)
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:1578.0,[[<frame.Event object at 0x0000023723136FD0>]]
(0, PairChainLv0, NEW)
Pattern01Lv0.updateAll()
test:PP.cursor()__start_cursor:[0, 84],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[0, 85],layer_chain:0,w:2
test:PP.cursor()__start_cursor:[1, 89],layer_chain:1,w:2
test:PP.cursor()__start_cursor:[1, 90],layer_chain:1,w:2
test:PP.cursor()__start_curs

In [24]:
[[st,st.mp,st.pp] for st in m.Lv2_L]

[[Trend2(1, 2, Point(0.0, 16.31, 0)), [0, 1, 2], 1],
 [Trend2(-1, 2, Point(14.0, 16.46, -1)), [1, 2, 3, 4, 5, 6, 7], 3],
 [Trend2(1, 2, Point(98.0, 15.76, 1)), [4, 5, 6, 7, 8], 3],
 [Trend2(-1, 2, Point(158.0, 16.34, -1)), [7, 8, 9, 10, 11], 3],
 [Trend2(1, 2, Point(227.0, 15.51, 1)), [10, 11, 12], 1],
 [Trend2(-1, 2, Point(275.0, 16.87, -1)), [11, 12, 13, 14, 15], 3],
 [Trend2(1, 2, Point(368.0, 14.74, 1)),
  [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
  7],
 [Trend2(-1, 2, Point(541.0, 18.07, -1)), [21, 22, 23, 24, 25], 1],
 [Trend2(1, 2, Point(582.0, 16.46, 1)), [22, 23, 24, 25, 26, 27, 28], 3],
 [Trend2(-1, 2, Point(631.0, 17.62, -1)), [25, 26, 27, 28, 29], 3],
 [Trend2(1, 2, Point(738.0, 15.88, 1)),
  [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
  7],
 [Trend2(-1, 2, Point(1031.0, 27.87, -1)),
  [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
  11],
 [Trend2(1, 2, Point(1408.0, 19.46, 1)), [46, 47, 48, 49, 50, 51, 52], 3],
 [Trend2(-1, 2, Point(1470.0, 20.

In [27]:
[[op.TmSig, op.TmOp, op.openV, op.pattern_objs] for op in m.position]

[[131.0,
  133.0,
  16.0,
  [Pair1(TmS:112.0, -1, [15, 16]), Pair1(TmS:117.0, -1, [17, 18])]],
 [134.0,
  141.0,
  16.03,
  [Pair1(TmS:112.0, -1, [15, 16]), Pair1(TmS:117.0, -1, [17, 18])]],
 [142.0,
  149.0,
  16.0,
  [Pair1(TmS:117.0, -1, [17, 18]),
   Pair1(TmS:129.0, -1, [19, 20, 21, 22, 23])]],
 [890.0,
  893.0,
  18.93,
  [Pair0(TmS:870.0, 1, [144, 145, 146, 147]),
   Pair0(TmS:880.0, 1, [148, 149])]],
 [894.0,
  908.0,
  19.39,
  [Pair0(TmS:870.0, 1, [144, 145, 146, 147]),
   Pair0(TmS:880.0, 1, [148, 149])]]]

In [ ]:
m.PCLv0_L[0].cL

In [24]:
# print Pair list
dd = m.PLv1_L[1].calFeatures()
dd['TmS'] = []
dd['chain_layer'] = []
dd['index'] = []
dd['S'] =[]
dd['P'] = []
for k in dd.keys():
    dd[k] = []
for p in m.PLv1_L:
    print(p)
    r = p.calFeatures()
    r['TmS'] = p.TmS
    r['chain_layer'] = p.chain_layer
    r['index'] = p.index
    r['S'] = p.S
    r['P'] = p.P
    for k in r.keys():
        dd[k].append(r[k])
df = pd.DataFrame(dd)
df = df[['TmS','chain_layer',  'index', 'drt','is_flag','cl', 'ch', 'a','k','c_c_level', 'c_drt'
         ,'S', 'P',  'b_tm', 'b_w', 'c_a',  'cc',
        'p_v', 's_tm', 's_v']]
df.to_excel('PairLv1.xlsx')

Pair-1(TmS:0.0, 0, [0])
Pair0(TmS:0.0, 1, [0])
Pair0(TmS:14.0, -1, [1, 2, 3, 4, 5, 6, 7, 8])
Pair1(TmS:48.0, 1, [2, 3, 4, 5, 6, 7, 8])
Pair2(TmS:75.0, -1, [3, 4, 5, 6, 7, 8])
Pair0(TmS:217.0, -1, [9])
Pair0(TmS:227.0, 1, [10, 11, 12])
Pair1(TmS:275.0, -1, [11, 12])
Pair2(TmS:307.0, 1, [12])
Pair0(TmS:312.0, -1, [13, 14, 15])
Pair1(TmS:368.0, 1, [14, 15])
Pair2(TmS:396.0, -1, [15])
Pair0(TmS:424.0, 1, [16, 17, 18, 19, 20, 21, 22])
Pair1(TmS:464.0, -1, [17, 18, 19, 20, 21, 22])
Pair2(TmS:491.0, 1, [18, 19])
Pair2(TmS:520.0, 1, [20])
Pair2(TmS:541.0, -1, [21, 22])
Pair0(TmS:594.0, -1, [23])
Pair0(TmS:609.0, 1, [24, 25, 26, 27, 28, 29])
Pair1(TmS:631.0, -1, [25, 26, 27, 28, 29])
Pair2(TmS:661.0, 1, [26, 27, 28, 29])
Pair0(TmS:814.0, 1, [30, 31, 32, 33])
Pair1(TmS:840.0, -1, [31, 32, 33])
Pair2(TmS:845.0, 1, [32, 33])
Pair0(TmS:984.0, 1, [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])
Pair1(TmS:1031.0, -1, [35, 36, 37, 38, 39, 40, 41, 42, 43, 44])
Pair2(TmS:1041.0, 1, [36, 37, 38, 39, 40, 41,

In [ ]:
[p.index for p in m.PLv1_L]

In [ ]:
[st.distr() for st in m.Lv1_L[5].stick_stack]

In [ ]:
m.Lv1_L[1].stick_stack[0]

In [ ]:
m.Lv1_L[1].stick_stack[0].peak

In [ ]:
m.Lv1_L[1].stick_stack[0].distr()

In [ ]:
l = [[st.drt,st.amp()[0], st.amp()[1]] for st in m.Lv1_L]
df = pd.DataFrame(l)
df.to_excel('00lv_amp_by_st.xlsx')

In [ ]:
tt = [list(st.distr()[0]) + list(st.distr()[1]) for st in m.Lv0_L[132:135]]
tt_df = pd.DataFrame(tt)
tt_df.to_excel('stdistr.xlsx')

In [ ]:
x = m.PLv0_L[15].calFeatures()
op = Operator(x,3,'pair_drt')
op2 = Operator(x,0.5,'concentrated')
r = Node([op,op2],1)
r.do()

In [ ]:
EventFactory.event_config

In [ ]:
m.findList('pair',0).__name__

In [ ]:
d = {'level_num':0, 'obj_name': 'Stick', 'event_name':'NEW', 'obj_p':'m.position[0]', 'method':'check_event', 'param':''}
EventFactory.reg_event(d)

In [ ]:
EventFactory.remove_event(d)

In [ ]:
# print Pair Lv0
dd = m.PLv0_L[5].output()
for k in dd.keys():
    dd[k] = []
for p in m.PLv0_L:
    r = p.output()
    for k in r.keys():
        dd[k].append(r[k])
df = pd.DataFrame(dd)
df.to_excel('pair0.xlsx')

In [25]:
# 把Lv0打印出来
dd = [[st.drt,st.start.TmIdx,st.start.V,st.peak.TmIdx, st.peak.V] for st in m.Lv0_L]
df_dd = pd.DataFrame(dd)
df_dd.to_excel('Lv0_L.xlsx')

In [26]:
# 把Lv1打印出来
dd = [[st.drt,st.start.TmIdx,st.start.V,st.peak.TmIdx, st.peak.V,st.pp,st.mp] for st in m.Lv1_L]
df_dd = pd.DataFrame(dd)
df_dd.to_excel('Lv1_L.xlsx')

In [ ]:
# 把Lv2打印出来
dd = [[st.drt,st.start.TmIdx,st.start.V,st.peak.TmIdx, st.peak.V,st.pp,st.mp] for st in m.Lv2_L]
df_dd = pd.DataFrame(dd)
df_dd.to_excel('Lv2_L.xlsx')

In [ ]:
st = m.Lv0_L[3]
k_L = [[k[1],k[2]] for k in dt if k[4]>= st.start.TmIdx and k[4]<= st.peak.TmIdx]

interval = (st.peak.V - st.start.V)*st.drt/ 20
L = min(st.peak.V, st.start.V)
H = max(st.peak.V, st.start.V)
scale = [L]
for i in list(range(1,20)):
    scale.append(scale[i-1]+interval)
scale.append(H)

distr = np.zeros(20)
for k in k_L:
    distr_k = np.zeros(20)
    for i in list(range(20)):
        if (min(k) <= scale[i] <= max(k)) or (min(k) <= scale[i+1] <= max(k)):
            distr_k[i] = 1
    cnt = distr_k.sum()
    distr_k = distr_k/cnt
    distr = distr + distr_k
distr, distr.argmax()

In [ ]:
ll = np.zeros(20)
ll[2:4] = 1
cnt = ll.sum()
ll = ll/cnt

ll2 = np.ones(20)
lr = ll +ll2
lr

In [ ]:
a = [1,2,3]
a[:-1]

### draw and verified

In [ ]:
# 绘图用插值函数
# input: 
#       1 - TmIdx array 
#       2 - Values array
# output: 按x轴查完值的value array
def interpoint(TmIdx_array, V_array):
    global x
    x['TmIdx'].astype('int64')
    print(len(x))
    df_tmp = pd.DataFrame({'V': V_array, 'TmIdx': TmIdx_array})
    df_tmp['TmIdx'].astype('int64')
    df_tmp = df_tmp.merge(x, how='right', left_on='TmIdx', right_on='TmIdx')
    print(len(df_tmp))
    df_tmp.sort_values(['TmIdx'], inplace=True)
    return df_tmp['V'].interpolate().values.tolist()

In [ ]:
def draw(lv_list):
    global dt, interpoint
    x_idx = [k[4] for k in dt]
    oringe_dt = [[k[0], k[3], k[2], k[1]] for k in dt]

    n = len(lv_list)
    drawing_line = []
    
    for i in list(range(n)):
        tm, v = [], []
        for stick in lv_list[i]:
            tm.append(stick.start.TmIdx)
            v.append(stick.start.V)
        dt_line = interpoint(tm,v)
        line = Line('stick')
        line.add('xxx', [k[4] for k in dt], dt_line
        ,is_datazoom_show=True
        )  
        drawing_line.append(line)
        
    overlap = Overlap()
    for line in drawing_line:
        overlap.add(line)
    
    return overlap
            

In [ ]:
draw([Stick.lv_L, TrendLv1.lv_L, TrendLv2.lv_L])

In [ ]:
CenterStrict(Stick.lv_L[1:], market)

In [ ]:
for c in market.CenterStrict_LD[10]:
    print(c)

In [ ]:
a = [1,2,3,4,9]
l = [1,2,3,4,9]
for aa,ll in zip(*[a,l]):
    print(aa,ll)

In [ ]:
m.__class__.__name__

In [ ]:
a = []
a.append([])
a.append([])
a.append([])

In [ ]:
a

In [ ]:
tmp = type('Xb', (object,), dict(a=1))
tmp = type('Xa', (object,), dict(a=1))

In [ ]:
a = [1,2,3]
b = [3,4,5]
for aa in a [1:
             :-1]:
    print(aa)

In [ ]:
d1 = {'a': 1}
d2 = {'b': 2}

In [ ]:
d1.append(d2)